In [2]:
#import ale_py
#import shimmy
import gymnasium as gym
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Dataset
from torch.nn import functional as F
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import random
import copy
import time
import pickle
import torchvision
import matplotlib.pyplot as plt


In [170]:

class CNN(nn.Module):

    def __init__(self):
        super().__init__()   
        self.linear1 = nn.Linear(4, 128)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(128,128)
        self.activation = nn.ReLU()
        self.linear3 = nn.Linear(128,2)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.activation(x)
        x = self.linear3(x)
        
        return x


In [ ]:
class Memory():
    
    def __init__(self,size):
        self.size = size
        self.experiences = []
    
    def sample(self,batch_size, batch_number):
        data = []
        for _ in range(batch_number):
            batch = random.choices(self.experiences, k=batch_size)
            data.append(batch)
        return data
        
    def add(self, experience):
        self.experiences.append(experience)
        if len(self.experiences) > self.size:
            self.experiences.pop(0)

In [174]:
class DQN_agent:

    def __init__(self):
        # Get cpu, gpu or mps device for training.
        self.device = (
            "cuda"
            if torch.cuda.is_available()
            else "mps"
            if torch.backends.mps.is_available()
            else "cpu"
        )
        print(f"Using {self.device} device")
        self.pred_NN = CNN().to(self.device)
        self.target_NN = copy.deepcopy(self.pred_NN)
        self.target_NN.eval()
        self.optimizer = torch.optim.Adam(self.pred_NN.parameters(), lr=0.0001)
    
    def predict(self, x):
        return self.pred_NN.forward(x)
    
    def action(self, pred):
        return (
            random.randint(0, pred.size(dim=0) - 1)
            if random.random() < epsilon
            else torch.argmax(pred).item()
        )
    
    def train(self, experiences):
        data_set = ExperiencesDataset(experiences, self)
        data_loader = DataLoader(data_set, batch_size=samples_per_batch, shuffle=True)
        loss_fn = nn.MSELoss()
        epoch_loss = 0
        self.pred_NN.train()

        for (state, a, y) in data_loader:  
            self.optimizer.zero_grad()
            pred = self.pred_NN(state)
            indicies = torch.LongTensor(a)
            indicies =indicies.unsqueeze(dim=0).T
            pred = pred.gather(1,indicies)
            loss = loss_fn(y, pred)
            loss.backward()
            self.optimizer.step()
            epoch_loss += loss.item()

        self.pred_NN.train(mode=False)
        return epoch_loss
        
    def copy(self):
        self.target_NN.load_state_dict(self.pred_NN.state_dict())  
        
    def estimated_value(self, experience):
        pass

In [172]:
class ExperiencesDataset(Dataset):
    
    def __init__(self, experiences, agent):
    
        self.states = []
        self.labels = []
        self.actions = []
        for _, experience in enumerate(experiences):
            state, action, reward, next_state, done = experience
            with torch.no_grad():
                if done:
                    target = torch.tensor(reward)
                else:
                    target_pred = agent.target_NN.forward(next_state)
                    target = reward + gamma * target_pred.max()
            self.labels.append(target) 
            self.actions.append(action)
            self.states.append(state)
            
    def __len__(self):
        return len(self.states)
    
    def __getitem__(self, idx):
        return self.states[idx], self.actions[idx], self.labels[idx]

In [177]:
#initialize environment
env = gym.make("CartPole-v1",render_mode="rgb_array")
#preprocess environment
#env = gym.wrappers.AtariPreprocessing(env, screen_size=84, grayscale_obs=True, frame_skip=1, noop_max=30, scale_obs = True)
#start environment
#create action array
actions = range(env.action_space.n)

#hyperparams
epsilon = 0.2
epsilon_decay = 1
gamma = 0.95
max_steps = 9999
training_freq = 1
copying_freq = 100
samples_per_training_session = 128
samples_per_batch = 128

#initialize agent
agent = DQN_agent()
action = random.choice(actions)

training_session = 0
max_episode = 5000

#loops until max_time is reached
experiences = []
experiences_limit = 400
total_steps = 0
for episode  in range(1,max_episode):
    #get first states
    state, info = env.reset()
    #loops until experience_capacity is reached
    episode_reward = 0 
    episode_loss = 0
    steps = 0
    for i in range(1, max_steps):
        #predict q-values and choose action
        with torch.no_grad():
            pred = agent.predict(torch.tensor(state))
        action = agent.action(pred)
        #get next states
        next_state, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        if i == max_steps - 1:
            print("Max steps reached.")
        next_state = torch.from_numpy(next_state)
        save_this = [torch.tensor(state), action, reward, torch.tensor(next_state), terminated] 
        experiences.append(save_this)
        if len(experiences) >= experiences_limit:
            experiences.pop(0) 
        if terminated or truncated:
            break
        state = next_state      
        #train
        if total_steps % training_freq == 0 and len(experiences) > 2*samples_per_training_session:
            training_session += 1
            experiences_train = random.choices(experiences, k=samples_per_training_session)
            episode_loss += agent.train(experiences_train)
        total_steps += 1
        steps = i
    #if episode % 100 == 0:
    if episode % copying_freq == 0:
        agent.copy()
    print(f"Episode: {episode} Reward: {episode_reward} loss: {episode_loss/steps}")

Using cpu device
Episode: 1 Reward: 17.0 loss: 0.0
Episode: 2 Reward: 15.0 loss: 0.0
Episode: 3 Reward: 15.0 loss: 0.0
Episode: 4 Reward: 17.0 loss: 0.0


/tmp/ipykernel_26224/169441352.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  save_this = [torch.tensor(state), action, reward, torch.tensor(next_state), terminated]
/tmp/ipykernel_26224/169441352.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = agent.predict(torch.tensor(state))


Episode: 5 Reward: 17.0 loss: 0.0
Episode: 6 Reward: 15.0 loss: 0.0
Episode: 7 Reward: 16.0 loss: 0.0
Episode: 8 Reward: 16.0 loss: 0.0
Episode: 9 Reward: 14.0 loss: 0.0
Episode: 10 Reward: 19.0 loss: 0.0
Episode: 11 Reward: 20.0 loss: 0.0
Episode: 12 Reward: 19.0 loss: 0.0
Episode: 13 Reward: 20.0 loss: 0.0
Episode: 14 Reward: 17.0 loss: 0.0


/home/konrad/Documents/pyenv/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([128, 1])) that is different to the input size (torch.Size([128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Episode: 15 Reward: 21.0 loss: 0.049674248695373534
Episode: 16 Reward: 20.0 loss: 0.781154707858437
Episode: 17 Reward: 13.0 loss: 0.48521363983551663
Episode: 18 Reward: 15.0 loss: 0.2791349046996662
Episode: 19 Reward: 13.0 loss: 0.1220340970903635
Episode: 20 Reward: 17.0 loss: 0.0334179624915123
Episode: 21 Reward: 13.0 loss: 0.010167492475981513
Episode: 22 Reward: 14.0 loss: 0.010609699950481836
Episode: 23 Reward: 12.0 loss: 0.009502908299592409
Episode: 24 Reward: 13.0 loss: 0.010579091884816686
Episode: 25 Reward: 10.0 loss: 0.009249388033317195
Episode: 26 Reward: 11.0 loss: 0.00893229190260172
Episode: 27 Reward: 14.0 loss: 0.007200633784612784
Episode: 28 Reward: 14.0 loss: 0.006688649766147137
Episode: 29 Reward: 15.0 loss: 0.006749140830444438
Episode: 30 Reward: 13.0 loss: 0.00527234534577777
Episode: 31 Reward: 15.0 loss: 0.004877346334978938
Episode: 32 Reward: 12.0 loss: 0.003955902489410205
Episode: 33 Reward: 17.0 loss: 0.0031373991369036958
Episode: 34 Reward: 12.

KeyboardInterrupt: 

In [ ]:

print(f'*****stopped training after {elapsed_time} seconds*****\n')

print('*****Plot filters in first layer*****\n')
kernels = agent.pred_NN.stack[0].weight.detach().clone()
kernels = kernels - kernels.min()
kernels = kernels / kernels.max()
filter_img = torchvision.utils.make_grid(kernels, nrow = 8)
plt.imshow(filter_img.permute(1, 2, 0))
plt.show() 

print('*****Pickle Dumping model*****\n')

file = open('safetyPickleDump', 'wb')
pickle.dump(agent, file)
file.close()

print("*****Saving model*****")
torch.save(agent.pred_NN.state_dict(), "model")

env.close()

In [72]:
A = torch.tensor([[1,2],[3,4],[5,6],[7,8]])
indicies = torch.LongTensor([0,1,0,1])
indicies =indicies.unsqueeze(dim=0).T

In [73]:
indicies

tensor([[0],
        [1],
        [0],
        [1]])

In [74]:
A.gather(1,indicies)

tensor([[1],
        [4],
        [5],
        [8]])